## Data Wrangling / Capstone Project 1

Overview:

Preliminiary look:
- Check the shape and columns of 2017 trip data
- Check that data is 'tidy'(Rows represent individual observations and columns are separate variables). As each row contains      information for a unique trip, the index can be set trip_id
- Can use .concat() to join 2017 trip data

Merge df's:
- Join 2017 trip data into one df
- Combine station data 

Clean Data:
- Column names: make consistent 
- Fill in missing values 
- Data types: change ustertype and gender to category, start and end_times to datetime, and change birth_year to numberic
- Drop duplicate rows: count unique trip_id's 

Outliers
- Check to make sure category values make sense
- Trip duration
- Station capacity
- Rides per day

## Importing Modules

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

## Loading Data

In [2]:
Q1 = pd.read_csv('../../divvy_data/Divvy_Trips_2017_Q1.csv')


In [3]:
Q2 = pd.read_csv('../../divvy_data/Divvy_Trips_2017_Q2.csv')

In [4]:
Q3 = pd.read_csv('../../divvy_data/Divvy_Trips_2017_Q3.csv')

In [5]:
Q4 = pd.read_csv('../../divvy_data/Divvy_Trips_2017_Q4.csv')

In [6]:
#Q1-Q3 data is of format MM/DD/YYYY HH:MM:SS and putting the Q4 data in the same  formate will make it easier to convert 
#the dates into datetime objects later in the process

Q4.start_time = Q4.start_time +':00'
Q4.start_time.head()

0    12/31/2017 23:58:00
1    12/31/2017 23:54:00
2    12/31/2017 23:54:00
3    12/31/2017 23:48:00
4    12/31/2017 23:42:00
Name: start_time, dtype: object

In [7]:
Q4.end_time = Q4.end_time +':00'
Q4.end_time.head()

0       1/1/2018 0:03:00
1       1/1/2018 0:18:00
2       1/1/2018 0:18:00
3    12/31/2017 23:53:00
4    12/31/2017 23:47:00
Name: end_time, dtype: object

In [8]:
trips17 = pd.concat([Q1,Q2,Q3,Q4], ignore_index=True)
trips17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3829014 entries, 0 to 3829013
Data columns (total 12 columns):
trip_id              int64
start_time           object
end_time             object
bikeid               int64
tripduration         int64
from_station_id      int64
from_station_name    object
to_station_id        int64
to_station_name      object
usertype             object
gender               object
birthyear            float64
dtypes: float64(1), int64(5), object(6)
memory usage: 379.8+ MB


In [9]:
stations17 = pd.read_csv('../../divvy_data/Divvy_Stations_2017_Q3Q4.csv')
stations17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 585 entries, 0 to 584
Data columns (total 8 columns):
id             585 non-null int64
name           585 non-null object
city           585 non-null object
latitude       585 non-null float64
longitude      585 non-null float64
dpcapacity     585 non-null int64
online_date    585 non-null object
Unnamed: 7     0 non-null float64
dtypes: float64(3), int64(2), object(3)
memory usage: 41.1+ KB


In [10]:
#Merge station info with trip info
first = pd.merge(trips17, stations17, left_on='from_station_id', right_on='id', how='left')
data17 = pd.merge(first, stations17, left_on='to_station_id', right_on='id', how='left')

In [11]:
#Drop un-needed columns
data17 = data17.drop(['Unnamed: 7_x','Unnamed: 7_y','name_x','name_y','id_x', 'id_y'], axis=1)


In [12]:
#Rename columns
data17.columns = ['trip_id', 'start_time','end_time','bike_id','trip_duration','start_station_id','start_station_name', \
                  'end_station_id','end_station_name', 'user_type', 'gender', 'birth_year','start_city', 'start_latitude', \
                  'start_longitude', 'start_capacity', 'start_online_date','end_city','end_latitude','end_longitude', \
                  'end_capacity', 'end_online_date']
data17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3829014 entries, 0 to 3829013
Data columns (total 22 columns):
trip_id               int64
start_time            object
end_time              object
bike_id               int64
trip_duration         int64
start_station_id      int64
start_station_name    object
end_station_id        int64
end_station_name      object
user_type             object
gender                object
birth_year            float64
start_city            object
start_latitude        float64
start_longitude       float64
start_capacity        int64
start_online_date     object
end_city              object
end_latitude          float64
end_longitude         float64
end_capacity          int64
end_online_date       object
dtypes: float64(5), int64(7), object(10)
memory usage: 671.9+ MB


In [13]:
#Missing values: only two columns did not contain complete data. Gender and birth_year were only pulled for rides with a 
#user_type of subscriber. We will hold off on filling in birth_year once we change it to a nuermic data type.
data17.gender = data17.gender.fillna('Non Subscriber')
print len(data17.gender)


3829014


In [14]:
#Change the birth year columns froms strings to numbers
data17.birth_year = pd.to_numeric(data17.birth_year)

In [15]:
"""Take a look a the birth years to see if they make sense. It seems there is a fair number of riders who are listed as being over
100 years old. As its not very likely people of this age are riding bikes through Chicago, this information does not seem to be
accurate."""
data17.groupby('birth_year').count()

,trip_id,start_time,end_time,bike_id,trip_duration,start_station_id,start_station_name,end_station_id,end_station_name,user_type,...,start_city,start_latitude,start_longitude,start_capacity,start_online_date,end_city,end_latitude,end_longitude,end_capacity,end_online_date
birth_year,,,,,,,,,,,,,,,,,,,,,
1899,82,82,82,82,82,82,82,82,82,82,...,82,82,82,82,82,82,82,82,82,82
1900,179,179,179,179,179,179,179,179,179,179,...,179,179,179,179,179,179,179,179,179,179
1901,60,60,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
1906,84,84,84,84,84,84,84,84,84,84,...,84,84,84,84,84,84,84,84,84,84
1909,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
1910,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1915,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
1916,74,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74
1918,700,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700


In [16]:
"""Because birth_year seems to include inaccurate information, and is only available for Subscribers, it will not be very helpful
in grouping all riders by age. As the main purpose of the project will be to focus on station usage and proper stocking
requirements, we can remove this info from the dataframe"""

data17 = data17.drop('birth_year', axis=1)

In [17]:
#Because each row displays info for an individual trip, we can set the index to trip_id
data17 = data17.set_index('trip_id')

In [18]:
#change column types
data17.user_type = data17.user_type.astype('category')
data17.gender = data17.gender.astype('category')

In [19]:
#change start_time to datetime objects
data17['start_time'] = pd.to_datetime(data17['start_time'], format='%m/%d/%Y  %H:%M:%S')


In [20]:
data17.start_time.head()

trip_id
13518905   2017-03-31 23:59:07
13518904   2017-03-31 23:56:25
13518903   2017-03-31 23:55:33
13518902   2017-03-31 23:54:46
13518901   2017-03-31 23:53:33
Name: start_time, dtype: datetime64[ns]

In [21]:
#change end_time to datetime objects
data17['end_time'] = pd.to_datetime(data17['end_time'], format='%m/%d/%Y  %H:%M:%S')

In [22]:
data17.end_time.head()

trip_id
13518905   2017-04-01 00:13:24
13518904   2017-04-01 00:00:21
13518903   2017-04-01 00:01:21
13518902   2017-03-31 23:59:34
13518901   2017-04-01 00:00:28
Name: end_time, dtype: datetime64[ns]

In [23]:
#get rid of any duplicate rows
data17 = data17.drop_duplicates()

print data17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3828957 entries, 13518905 to 16734066
Data columns (total 20 columns):
start_time            datetime64[ns]
end_time              datetime64[ns]
bike_id               int64
trip_duration         int64
start_station_id      int64
start_station_name    object
end_station_id        int64
end_station_name      object
user_type             category
gender                category
start_city            object
start_latitude        float64
start_longitude       float64
start_capacity        int64
start_online_date     object
end_city              object
end_latitude          float64
end_longitude         float64
end_capacity          int64
end_online_date       object
dtypes: category(2), datetime64[ns](2), float64(4), int64(6), object(6)
memory usage: 562.3+ MB
None


## Checking for outliers:

In [24]:
#looking at the user_types to make sure categories make sense
data17.groupby('user_type').count()

,start_time,end_time,bike_id,trip_duration,start_station_id,start_station_name,end_station_id,end_station_name,gender,start_city,start_latitude,start_longitude,start_capacity,start_online_date,end_city,end_latitude,end_longitude,end_capacity,end_online_date
user_type,,,,,,,,,,,,,,,,,,,
Customer,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859,836859
Dependent,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
Subscriber,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091,2992091


In [25]:
#look into what dependent means. The readme only states the definition of subscribers(annual subscriptions) and customers
#(purchased a 24 hour pass)
data17.loc[data17.user_type == 'Dependent']

,start_time,end_time,bike_id,trip_duration,start_station_id,start_station_name,end_station_id,end_station_name,user_type,gender,start_city,start_latitude,start_longitude,start_capacity,start_online_date,end_city,end_latitude,end_longitude,end_capacity,end_online_date
trip_id,,,,,,,,,,,,,,,,,,,,
13349140,2017-03-04 13:26:20,2017-03-04 13:43:25,2569,1025,207,Emerald Ave & 28th St,207,Emerald Ave & 28th St,Dependent,Male,Chicago,41.843580,-87.645368,15,8/14/2013 10:26,Chicago,41.843580,-87.645368,15,8/14/2013 10:26
13313514,2017-02-27 07:02:01,2017-02-27 07:06:05,706,244,207,Emerald Ave & 28th St,206,Halsted St & Archer Ave,Dependent,Male,Chicago,41.843580,-87.645368,15,8/14/2013 10:26,Chicago,41.847203,-87.646795,15,8/14/2013 8:53
13231738,2017-02-18 07:47:21,2017-02-18 07:52:32,4576,311,207,Emerald Ave & 28th St,206,Halsted St & Archer Ave,Dependent,Male,Chicago,41.843580,-87.645368,15,8/14/2013 10:26,Chicago,41.847203,-87.646795,15,8/14/2013 8:53
13070858,2017-01-22 08:14:54,2017-01-22 08:18:51,3190,237,207,Emerald Ave & 28th St,206,Halsted St & Archer Ave,Dependent,Male,Chicago,41.843580,-87.645368,15,8/14/2013 10:26,Chicago,41.847203,-87.646795,15,8/14/2013 8:53
15777515,2017-08-13 13:05:00,2017-08-13 13:18:24,3570,804,177,Theater on the Lake,327,Sheffield Ave & Webster Ave,Dependent,Non Subscriber,Chicago,41.926277,-87.630834,23,7/19/2013 11:09,Chicago,41.921540,-87.653818,19,9/23/2013 15:05
15776556,2017-08-13 12:35:43,2017-08-13 13:04:52,3570,1749,177,Theater on the Lake,177,Theater on the Lake,Dependent,Non Subscriber,Chicago,41.926277,-87.630834,23,7/19/2013 11:09,Chicago,41.926277,-87.630834,23,7/19/2013 11:09
15776123,2017-08-13 12:20:38,2017-08-13 12:35:31,3570,893,327,Sheffield Ave & Webster Ave,177,Theater on the Lake,Dependent,Non Subscriber,Chicago,41.921540,-87.653818,19,9/23/2013 15:05,Chicago,41.926277,-87.630834,23,7/19/2013 11:09


In [26]:
#Because gender information is only available for subscribers, we can classify the user_type as a 'subscriber' if it has the 
#gender information and a 'customer' if it does 

dp = data17.loc[data17.user_type == 'Dependent']
dpi = list(dp.index)

for x in dpi:
    if data17.loc[x, 'gender'] == 'Non Subscriber':
        data17.loc[x,'user_type'] = 'Customer'
    else:
        data17.loc[x, 'user_type'] = 'Subscriber'

data17.groupby('user_type').count()


,start_time,end_time,bike_id,trip_duration,start_station_id,start_station_name,end_station_id,end_station_name,gender,start_city,start_latitude,start_longitude,start_capacity,start_online_date,end_city,end_latitude,end_longitude,end_capacity,end_online_date
user_type,,,,,,,,,,,,,,,,,,,
Customer,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862,836862
Dependent,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Subscriber,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095,2992095


In [27]:
#looking at the categories in gender to make sure they make sense
data17.groupby('gender').count()

,start_time,end_time,bike_id,trip_duration,start_station_id,start_station_name,end_station_id,end_station_name,user_type,start_city,start_latitude,start_longitude,start_capacity,start_online_date,end_city,end_latitude,end_longitude,end_capacity,end_online_date
gender,,,,,,,,,,,,,,,,,,,
Female,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631,746631
Male,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512,2245512
Non Subscriber,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814,836814


In [28]:
#Per the readme file for the 2017 data, the trip durations longer than one day or less than one minute have already been excluded
mins = data17.trip_duration / 60
mins.describe()

count    3828957.000000
mean          15.927338
std           26.157687
min            1.000000
25%            6.700000
50%           11.433333
75%           19.150000
max         1438.966667
Name: trip_duration, dtype: float64

In [29]:
# Check to see if there are any outliers in the station capacities
print 'Start stations'
print data17.start_capacity.describe()
print 'End stations'
print data17.end_capacity.describe()

Start stations
count    3828957.000000
mean          24.419968
std           10.049477
min            0.000000
25%           15.000000
50%           23.000000
75%           31.000000
max           55.000000
Name: start_capacity, dtype: float64
End stations
count    3828957.000000
mean          24.301290
std            9.993929
min            0.000000
25%           15.000000
50%           23.000000
75%           31.000000
max           55.000000
Name: end_capacity, dtype: float64


In [30]:
# Divvy stations can't have 0 capacity so we need to find which stations are listed this way.
print data17[['start_station_name', 'start_station_id']].loc[data17.start_capacity == 0].sort_values('start_station_id')
print data17[['end_station_name', 'end_station_id']].loc[data17.end_capacity == 0].sort_values('end_station_id')

                start_station_name  start_station_id
trip_id                                             
13378989  Commercial Ave & 83rd St               581
13230718  Commercial Ave & 83rd St               581
14750423  Commercial Ave & 83rd St               581
14583306  Commercial Ave & 83rd St               581
14519887  Commercial Ave & 83rd St               581
13991809  Commercial Ave & 83rd St               581
13991255  Commercial Ave & 83rd St               581
13764518  Commercial Ave & 83rd St               581
13764503  Commercial Ave & 83rd St               581
14353079    Phillips Ave & 82nd St               582
14345724    Phillips Ave & 82nd St               582
14345641    Phillips Ave & 82nd St               582
                  end_station_name  end_station_id
trip_id                                           
13230487  Commercial Ave & 83rd St             581
13150787  Commercial Ave & 83rd St             581
13118893  Commercial Ave & 83rd St             581
147

In [31]:
#the best option we have is to look at the capacity for these stations earlier in 2017
stationsQ2 = pd.read_csv('../../divvy_data/Divvy_Stations_2017_Q1Q2.csv', index_col='id')

print stationsQ2.dpcapacity.loc[581]
print stationsQ2.dpcapacity.loc[582]


15
11


In [32]:
#fill in the capacities from the first half of 2017
data17.start_capacity.loc[data17.start_station_id == 581] = 15
data17.end_capacity.loc[data17.end_station_id == 581] = 15
data17.start_capacity.loc[data17.start_station_id == 582] = 11
data17.end_capacity.loc[data17.end_station_id == 582] = 11

C:\Users\Danny\Miniconda2\envs\tutorial\lib\site-packages\pandas\core\indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
#final check to make sure the new capacities were assigned correctly
print 'Start stations'
print data17.start_capacity.describe()
print 'End stations'
print data17.end_capacity.describe()

Start stations
count    3828957.000000
mean          24.420012
std           10.049401
min           11.000000
25%           15.000000
50%           23.000000
75%           31.000000
max           55.000000
Name: start_capacity, dtype: float64
End stations
count    3828957.000000
mean          24.301338
std            9.993845
min           11.000000
25%           15.000000
50%           23.000000
75%           31.000000
max           55.000000
Name: end_capacity, dtype: float64


In [34]:
# Another potential outlier could be the number of trips that happened in one day
data17T = data17.reset_index()
data17T = data17T.set_index('start_time')

In [35]:
daily_trips = data17T.groupby(data17T.index.date).count()
print daily_trips

            trip_id  end_time  bike_id  trip_duration  start_station_id  \
2017-01-01     1727      1727     1727           1727              1727   
2017-01-02     1960      1960     1960           1960              1960   
2017-01-03     4537      4537     4537           4537              4537   
2017-01-04     3269      3269     3269           3269              3269   
2017-01-05     2917      2917     2917           2917              2917   
2017-01-06     2516      2516     2516           2516              2516   
2017-01-07     1330      1330     1330           1330              1330   
2017-01-08     1193      1193     1193           1193              1193   
2017-01-09     3816      3816     3816           3816              3816   
2017-01-10     3310      3310     3310           3310              3310   
2017-01-11     3821      3821     3821           3821              3821   
2017-01-12     3071      3071     3071           3071              3071   
2017-01-13     3676      

In [37]:
daily_trips.start_station_name.sort_values()

2017-12-25      240
2017-12-24      403
2017-12-30      548
2017-12-31      651
2017-12-26      779
2017-12-29     1076
2017-12-27     1138
2017-01-08     1193
2017-12-28     1294
2017-01-07     1330
2017-01-16     1406
2017-10-14     1563
2017-04-30     1622
2017-01-29     1623
2017-11-18     1664
2017-03-25     1675
2017-12-23     1727
2017-01-01     1727
2017-03-14     1746
2017-12-09     1790
2017-04-05     1845
2017-11-23     1851
2017-04-29     1879
2017-02-25     1944
2017-01-02     1960
2017-02-04     2103
2017-11-12     2152
2017-01-14     2160
2017-01-15     2195
2017-12-10     2200
              ...  
2017-06-21    18728
2017-08-11    18842
2017-08-23    18906
2017-08-25    18930
2017-08-14    19029
2017-08-07    19062
2017-06-03    19225
2017-07-25    19343
2017-07-28    19401
2017-07-31    19406
2017-06-10    19413
2017-07-02    19489
2017-08-09    19515
2017-07-17    19553
2017-08-18    19789
2017-07-18    19888
2017-06-24    20001
2017-07-01    20080
2017-08-02    20120
